In [1]:
#CSC620
#HA 11.1
#Paula Abigail Tam
# --- Program 0 ---

import math
import pandas as pd
import numpy as np

In [2]:
#This is the toy corpus with 3 documents
#and the query string that we will check for

#documents
doc1 = "I want to start learning to charge something in life"
doc2 = "reading something about life no one else knows"
doc3 = "Never stop learning"
#query string
query = "life learning"

In [3]:
#term -frequenvy :word occurences in a document
def compute_tf(docs_list):
    for doc in docs_list: #for every document in the list
        doc1_lst = doc.split(" ") #split by whitespaces
        wordDict_1= dict.fromkeys(set(doc1_lst), 0) #make a dictionary using the set of words in the doc as the key

        for token in doc1_lst: #for every word in the document
            wordDict_1[token] +=  1 #increment the count of that word by 1, since it's a dictionary, it'll count per word
        df = pd.DataFrame([wordDict_1])
        idx = 0
        new_col = ["Term Frequency"]    
        df.insert(loc=idx, column='Document', value=new_col)
        print(df) #print dataframe
        
compute_tf([doc1, doc2, doc3])

         Document  something  in  learning  I  want  start  to  life  charge
0  Term Frequency          1   1         1  1     1      1   2     1       1
         Document  something  about  one  else  knows  no  life  reading
0  Term Frequency          1      1    1     1      1   1     1        1
         Document  Never  learning  stop
0  Term Frequency      1         1     1


In [4]:
#Normalized Term Frequency
def termFrequency(term, document):
    normalizeDocument = document.lower().split() #make all the words lowercase
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument)) #count the term over the total length of the document

def compute_normalizedtf(documents):
    tf_doc = [] #list to store df stuff
    for txt in documents:
        sentence = txt.split() #split the document into words
        norm_tf= dict.fromkeys(set(sentence), 0) #dictionary to keep count
        for word in sentence:
            norm_tf[word] = termFrequency(word, txt) #gives relative frequency instead of just count
        tf_doc.append(norm_tf)
        df = pd.DataFrame([norm_tf])
        idx = 0
        new_col = ["Normalized TF"]    
        df.insert(loc=idx, column='Document', value=new_col)
        print(df) #print the df
    return tf_doc #actually returning the list not the dataframe

tf_doc = compute_normalizedtf([doc1, doc2, doc3])

        Document  something   in  learning    I  want  start   to  life  \
0  Normalized TF        0.1  0.1       0.1  0.1   0.1    0.1  0.2   0.1   

   charge  
0     0.1  
        Document  something  about    one   else  knows     no   life  reading
0  Normalized TF      0.125  0.125  0.125  0.125  0.125  0.125  0.125    0.125
        Document     Never  learning      stop
0  Normalized TF  0.333333  0.333333  0.333333


In [5]:
def inverseDocumentFrequency(term, allDocuments):
    numDocumentsWithThisTerm = 0
    for doc in range (0, len(allDocuments)): #https://www.w3schools.com/python/ref_func_range.asp
        #range() makes a sequence with a starting number (0) and an stopping point (len(allDocuments))
        #in this case, it acts as the index, goes from 0 to len(allDocuments)-1
        if term.lower() in allDocuments[doc].lower().split():
            numDocumentsWithThisTerm = numDocumentsWithThisTerm + 1 #increment count if the term is in the document
 
    if numDocumentsWithThisTerm > 0:
        return 1.0 + math.log(float(len(allDocuments)) / numDocumentsWithThisTerm) #log the value of allDocuments over count
    else:
        return 1.0 #or just return 1
    
def compute_idf(documents):
    idf_dict = {} #dictionary
    for doc in documents: #for every document
        sentence = doc.split() #for every sentence per document
        for word in sentence: #for every word in the sentence
            idf_dict[word] = inverseDocumentFrequency(word, documents) #count the idf for each word
    return idf_dict #return the idf dictionary
idf_dict = compute_idf([doc1, doc2, doc3])

compute_idf([doc1, doc2, doc3]) #idk why they didnt do print(idf_dict), but this shows the whole dictionary

{'I': 2.09861228866811,
 'want': 2.09861228866811,
 'to': 2.09861228866811,
 'start': 2.09861228866811,
 'learning': 1.4054651081081644,
 'charge': 2.09861228866811,
 'something': 1.4054651081081644,
 'in': 2.09861228866811,
 'life': 1.4054651081081644,
 'reading': 2.09861228866811,
 'about': 2.09861228866811,
 'no': 2.09861228866811,
 'one': 2.09861228866811,
 'else': 2.09861228866811,
 'knows': 2.09861228866811,
 'Never': 2.09861228866811,
 'stop': 2.09861228866811}

In [6]:
# tf-idf score across all docs for the query string("life learning")
def compute_tfidf_with_alldocs(documents , query): #comparing the document and the query
    tf_idf = []
    index = 0
    query_tokens = query.split() #the words we are comparing
    df = pd.DataFrame(columns=['doc'] + query_tokens) #create dataframe w/ queries as the columns
    for doc in documents:
        df['doc'] = np.arange(0 , len(documents))
        doc_num = tf_doc[index]
        sentence = doc.split() #split into sentences
        for word in sentence: 
            for text in query_tokens: #for every word in the sentence, compare with query
                if(text == word):
                    idx = sentence.index(word)
                    tf_idf_score = doc_num[word] * idf_dict[word]
                    tf_idf.append(tf_idf_score)
                    df.iloc[index, df.columns.get_loc(word)] = tf_idf_score
        index += 1 #increment df index
    df.fillna(0 , axis=1, inplace=True) #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html
    #filling in any NaN values with 0
    return tf_idf , df #tfidf is the list, df is the dataframe version
            
documents = [doc1, doc2, doc3]
tf_idf , df = compute_tfidf_with_alldocs(documents , query)
print(df)
#print(tf_idf)

   doc      life  learning
0  0.0  0.140547  0.140547
1  1.0  0.175683  0.000000
2  2.0  0.000000  0.468488


In [7]:
#from IPython.display import Image
#Image("../input/tfidf-kernel/cosinesimilarity.jpg")
#In the OG code, this image was used to show an example of the queries as vectors.

In [8]:
#Normalized TF for the query string("life learning")
def compute_query_tf(query): 
    query_norm_tf = {}
    tokens = query.split()
    for word in tokens:
        query_norm_tf[word] = termFrequency(word , query) 
    return query_norm_tf
query_norm_tf = compute_query_tf(query)
print(query_norm_tf)

#this function gives us the term frequency of just the query string
#e.g. if query string("life learning is fun")
#output would be {'life': 0.25, 'learning': 0.25, 'is': 0.25, 'fun': 0.25}

{'life': 0.5, 'learning': 0.5}


In [13]:
#idf score for the query string("life learning")
def compute_query_idf(query):
    idf_dict_qry = {}
    sentence = query.split()
    documents = [doc1, doc2, doc3] #the documents are hard-coded here since it is just a toy corpus
    #print(documents)
    for word in sentence:
        idf_dict_qry[word] = inverseDocumentFrequency(word ,documents)
    return idf_dict_qry
idf_dict_qry = compute_query_idf(query)
print(idf_dict_qry)

#this function gives us the idf score of just the query string comparing with the documents

{'life': 1.4054651081081644, 'learning': 1.4054651081081644}


In [10]:
#tf-idf score for the query string("life learning")
def compute_query_tfidf(query):
    tfidf_dict_qry = {}
    sentence = query.split()
    for word in sentence:
        tfidf_dict_qry[word] = query_norm_tf[word] * idf_dict_qry[word]
        #this function uses the two previous functions and multiplies the results together to get the tfidf score
    return tfidf_dict_qry
tfidf_dict_qry = compute_query_tfidf(query)
print(tfidf_dict_qry)

{'life': 0.7027325540540822, 'learning': 0.7027325540540822}


In [11]:
#Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

"""
Example : Dot roduct(Query, Document1) 

     life:
     = tfidf(life w.r.t query) * tfidf(life w.r.t Document1) +  / 
     sqrt(tfidf(life w.r.t query)) * 
     sqrt(tfidf(life w.r.t doc1))
     
     learning:
     =tfidf(learning w.r.t query) * tfidf(learning w.r.t Document1)/
     sqrt(tfidf(learning w.r.t query)) * 
     sqrt(tfidf(learning w.r.t doc1))

"""
def cosine_similarity(tfidf_dict_qry, df , query , doc_num):
    dot_product = 0
    qry_mod = 0
    doc_mod = 0
    tokens = query.split()
   
    for keyword in tokens:
        dot_product += tfidf_dict_qry[keyword] * df[keyword][df['doc'] == doc_num] #the dot product of the query and document
        #||Query||
        qry_mod += tfidf_dict_qry[keyword] * tfidf_dict_qry[keyword]
        #||Document||
        doc_mod += df[keyword][df['doc'] == doc_num] * df[keyword][df['doc'] == doc_num]
        #to get the absolute value of both the queries and documents
        #first the og poster ^2 the value and then square root the result to get a positive value
    qry_mod = np.sqrt(qry_mod)
    doc_mod = np.sqrt(doc_mod)
    #implement formula
    denominator = qry_mod * doc_mod #this is the denominator of the cosine similarity
    cos_sim = dot_product/denominator #this is the whole cosine similarity
     
    return cos_sim

from collections.abc import Iterable
#changed the import from collections to collections.abc
#since Iterable from collections import was deprecated
def flatten(lis): #this function is just to flatten a list
     for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item): #recursive call
                yield x
        else:        
             yield item #yeild can return a sequence of values

In [12]:
def rank_similarity_docs(data):
    cos_sim =[]
    for doc_num in range(0 , len(data)):
        cos_sim.append(cosine_similarity(tfidf_dict_qry, df , query , doc_num).tolist())
    return cos_sim
similarity_docs = rank_similarity_docs(documents)
doc_names = ["Document1", "Document2", "Document3"]
print(doc_names)
print(list(flatten(similarity_docs))) #need to flatten the list since similarity_docs is a list of lists

['Document1', 'Document2', 'Document3']
[1.0, 0.7071067811865475, 0.7071067811865475]
